In [21]:
import pandas as pd
import numpy as np
import nltk
import re
import os
from wordcloud import WordCloud
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import verbnet
from gensim import corpora, models

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Exercise 1

In [4]:
filename = os.path.join('hillary-clinton-emails','Emails.csv') 
df = pd.read_csv(filename)
df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...


We decided to work only with the the ExtractedBodyText even if the rawText could have gave us more information as it was advise on slack.

In [5]:
extractedBodyText = df['ExtractedBodyText']
extractedBodyText.head()

0                                                  NaN
1    B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...
2                                                  Thx
3                                                  NaN
4    H <hrod17@clintonemail.com>\r\nFriday, March 1...
Name: ExtractedBodyText, dtype: object

In [6]:
# Dealing with Nan values
extractedBodyText.fillna("", inplace = True)

In [7]:
# Creation of the string we will use for the worldcloud
theText = ""
for i in extractedBodyText.index:
    theText = theText + "".join(extractedBodyText[i])

In [9]:
wordcloud = WordCloud().generate(theText)

In [10]:
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(theText)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## Cleaning of the data

In [12]:
#We remove
extractedBodyText2 = extractedBodyText.apply(lambda x: re.sub('<.*>', ' ', x ))
del extractedBodyText

In [15]:
extractedBodyText3 = extractedBodyText2.apply(lambda x: re.sub('030311 .* \\n', '', x))
del extractedBodyText2

In [16]:
extractedBodyText4 = extractedBodyText3.apply(lambda x: re.sub('^STATE DEPT .* STATE-[0-9A-Z]+', ' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Date: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Doc No.: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Case No.: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Sent: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('To: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('^[A-Z] .* @ .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('[A-Z]{1}[a-z]+, [A-Z]{1}[a-z]+ \d+, [0-9]+ \d+:\d+ [A-Z]{2}', ' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub(' [a-zA-Z]{1} ', ' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub(' [a-zA-Z]{2} ', ' ', x))
del extractedBodyText3

In [17]:
extractedBodyText5 = extractedBodyText4.apply(lambda x: re.sub('\n', ' ', x))
del extractedBodyText4

In [19]:
fifthAttempt = ""
for i in extractedBodyText5.index:
    fifthAttempt = fifthAttempt + "".join(extractedBodyText5[i])

In [20]:
stopwords = nltk.corpus.stopwords.words('english')
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
noStopWords = [word for word in nltk.word_tokenize(fifthAttempt) if word not in stopwords]
englishWords = [word for word in noStopWords if word in english_vocab]

In [22]:
noVerbs = [word for word in englishWords if word not in verbnet.lemmas()]

In [23]:
noVerbsString = ""
for i in range(len(noVerbs)):
    noVerbsString = noVerbsString + " " + noVerbs[i]

In [24]:
wordcloud6 = WordCloud().generate(noVerbsString)
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud6)
plt.axis("off")

# lower max_font_size
wordcloud6 = WordCloud(max_font_size=40).generate(noVerbsString)
plt.figure()
plt.imshow(wordcloud6)
plt.axis("off")
plt.show()

# Preparing the corpus

### Stopwords, stemming and tokenizing
#### 1. load list of English stop words

In [27]:
stopwords =  nltk.corpus.stopwords.words('english')

In [28]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

Import the Snowball Stemmer which is actually part of NLTK
<br>We should do **stemming** - breaking a word down into its root.</br>

In [30]:
stemmer = SnowballStemmer("english")

In [25]:
email_receivers_file = os.path.join('hillary-clinton-emails', 'EmailReceivers.csv')
emails_file = os.path.join('hillary-clinton-emails', 'Emails.csv')
persons_file = os.path.join('hillary-clinton-emails', 'Persons.csv')

In [3]:
email_receivers_df = pd.read_csv(email_receivers_file)
email_receivers_df.head(1)

,Id,EmailId,PersonId
0,1,1,80


In [4]:
emails_df = pd.read_csv(emails_file)
emails_df.head(2)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...


In [6]:
topic_modeling_df = emails_df.copy()
topic_modeling_df.drop(['Id'
                        , 'DocNumber'
                        , 'MetadataTo'
                        , 'MetadataFrom'
                        , 'SenderPersonId'
                        , 'MetadataDateSent'
                        , 'MetadataDateReleased'
                        , 'MetadataPdfLink'
                        , 'MetadataCaseNumber'
                        , 'ExtractedTo'
                        , 'ExtractedFrom'
                        , 'ExtractedCc'
                        , 'ExtractedDateSent'
                        , 'ExtractedCaseNumber'
                        , 'ExtractedDocNumber'
                        , 'ExtractedDateReleased'
                        , 'ExtractedReleaseInPartOrFull'], axis=1, inplace=True)
topic_modeling_df.head()

,MetadataSubject,MetadataDocumentClass,ExtractedSubject,ExtractedBodyText,RawText
0,WOW,HRC_Email_296,FW: Wow,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
2,CHRIS STEVENS,HRC_Email_296,Re: Chris Stevens,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
3,CAIRO CONDEMNATION - FINAL,HRC_Email_296,FVV: Cairo Condemnation - Final,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
4,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...


In [7]:
topic_modeling_df.shape

(7945, 5)

In [8]:
topic_modeling_df = topic_modeling_df.dropna(subset=['ExtractedBodyText'], how='all')

In [9]:
topic_modeling_df.head()

,MetadataSubject,MetadataDocumentClass,ExtractedSubject,ExtractedBodyText,RawText
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
2,CHRIS STEVENS,HRC_Email_296,Re: Chris Stevens,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
4,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...
5,MEET THE RIGHT-WING EXTREMIST BEHIND ANTI-MUSL...,HRC_Email_296,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\r\n-•-...-^\r\nH < hrod17@clintoner...,B6\r\nUNCLASSIFIED\r\nU.S. Department of State...
7,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.corn>\r\nFriday, March ...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...


In [545]:
re_longer_than_2 = re.compile('^[a-z]{4,}')

def preprocess_msg(msg):
    sentences = nltk.sent_tokenize(msg)
    del sentences[:6]
    del sentences[-7:]
    
    tokens = []
    for s in sentences:
        curr_tokens = nltk.word_tokenize(s)
        curr_tokens = [word for word in curr_tokens if word in english_vocab]
        curr_tokens = [word for word in curr_tokens if word not in stopwords]
        curr_tokens = list(filter(lambda x: re_longer_than_2.match(x) ,curr_tokens))
        tokens = tokens+curr_tokens

    return tokens    

### Stemming

In [533]:
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

def steamming(tokens_per_msg):
    return [[p_stemmer.stem(t) for t in tokens] for tokens in tokens_per_msg]

In [585]:
def calculate_topics(message_group):
    tokens_per_msg = [preprocess_msg(msg) for msg in message_group]
    #tokens_per_msg = steamming(tokens_per_msg)
    dictionary = corpora.Dictionary(tokens_per_msg)
    if not dictionary:
        return None
    #dictionary.filter_extremes(no_below=1, no_above=0.8)
    corpus = [dictionary.doc2bow(tokens) for tokens in tokens_per_msg]
    N = 5 if len(message_group) == 1 else 25
    ldamodel = models.ldamodel.LdaModel(corpus, num_topics=25, id2word = dictionary, passes=N)

    #topics_matrix
    return ldamodel.show_topics(formatted=False, num_words=5)    

In [586]:
topic_modeling_df['MsgTopics'] = ""
topic_modeling_df.head(1)

,MetadataSubject,MetadataDocumentClass,ExtractedSubject,ExtractedBodyText,RawText,MsgTopics
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...,


In [577]:
grouped_msgs_by_subject = topic_modeling_df.groupby(['MetadataSubject'])
gb = grouped_msgs_by_subject.groups

In [587]:
t = 0
for key, indices in gb.items():
    group = grouped_msgs_by_subject.get_group(key)
    msg_dict = group['RawText'].to_dict()
    topics_matrix = calculate_topics(list(msg_dict.values()))
    
    topics_list = "" if not topics_matrix else ' '.join([w for w, p in topics_matrix[0][1]])
    
    #topics_list = "" if not topics_matrix else [' '.join([w for w, p in words]) for i, words in topics_matrix]
    t += 1
    print(t)
    for i in indices:
        topic_modeling_df.ix[i, 'MsgTopics'] = topics_list

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [588]:
topic_modeling_df.head()

,MetadataSubject,MetadataDocumentClass,ExtractedSubject,ExtractedBodyText,RawText,MsgTopics
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...,military support zone chief commander
2,CHRIS STEVENS,HRC_Email_296,Re: Chris Stevens,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...,stand front former morning terribly
4,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...,military support zone chief commander
5,MEET THE RIGHT-WING EXTREMIST BEHIND ANTI-MUSL...,HRC_Email_296,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\r\n-•-...-^\r\nH < hrod17@clintoner...,B6\r\nUNCLASSIFIED\r\nU.S. Department of State...,print
7,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.corn>\r\nFriday, March ...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...,military support zone chief commander


In [ ]:
topic_modeling_df

In [490]:
type(msgs)

pandas.core.series.Series

In [10]:
topic_modeling_dict = topic_modeling_df.to_dict()

In [11]:
extracted_body_text_list = list(topic_modeling_df['ExtractedBodyText'])
len(extracted_body_text_list)

6742

In [12]:
extracted_body_text_list[:2]

['B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: Latest How Syria is aiding Qaddafi and more... Sid\r\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\r\nMarch 3, 2011\r\nFor: Hillary',
 'Thx']

In [13]:
# remove one-word texts
extracted_body_text_list = list(filter(lambda body: ' ' in body, extracted_body_text_list))
len(extracted_body_text_list)

5938

In [109]:
extracted_body_text_list[:2]

['B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: Latest How Syria is aiding Qaddafi and more... Sid\r\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\r\nMarch 3, 2011\r\nFor: Hillary',
 'H <hrod17@clintonemail.com>\r\nFriday, March 11, 2011 1:36 PM\r\nHuma Abedin\r\nFw: H: Latest: How Syria is aiding Qaddafi and more... Sid\r\nhrc memo syria aiding libya 030311.docx\r\nPis print.']

In [110]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [111]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [112]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

In [130]:
filter_rules = ['<.*>'
          , '^STATE DEPT .* STATE-[0-9A-Z]+'
          , 'Date: .* \\n'
          , 'Doc No.: .* \\n'
          , 'Case No.: .* \\n'
          , 'Sent: .* \\n'
          , 'To: .* \\n'
          , '^[A-Z] .* @ .* \\n'
          , '[^a-z] :.[A-Z]'
          , '[A-Z]{1}[a-z]+, [A-Z]{1}[a-z]+ \d+, [0-9]+ \d+:\d+ [A-Z]{2}'
          , '^[a-zA-Z]{1,2}$']

def apply_filters(series):
    for rule in filter_rules:
        series = series.apply(lambda x: re.sub(rule, '', x))
    return series

In [132]:
extracted_body_series = topic_modeling_df['ExtractedBodyText']

In [119]:
extracted_body_series.size

6742

In [120]:
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
months = ['january'
          , 'february'
          , 'march'
          , 'april'
          , 'may'
          , 'june'
          , 'july'
          , 'august'
          , 'september'
          , 'october'
          , 'november'
          , 'december']

In [134]:
extracted_body_series = apply_filters(extracted_body_series)

In [169]:
from nltk.stem.porter import *
stemmer1 = PorterStemmer()

In [172]:
def tokens_from_text(text):
    tokens = tokenize_only(text)
    tokens = [word for word in tokens if word not in stopwords]
    tokens = list(filter(lambda w: w not in days, tokens))
    tokens = list(filter(lambda w: w not in months, tokens))
    tokens = [word for word in tokens if word in english_vocab]
    return [stemmer1.stem(t) for t in tokens]

In [240]:
msg1 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739545 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN FULL From: Sullivan, Jacob J <Sullivan11@state.gov> Sent: Wednesday, September 12, 2012 10:16 AM To: Subject: FW: Wow From: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi Sent: Wednesday, September 12, 2012 10:09 AM To: Sullivan, Jacob J Subject: Wow What a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me too UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739545 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247"
msg2 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739546 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B6 From: Sent: To: Subject: Attachments: B6 Thursday, March 3, 2011 9:45 PM H: Latest How Syria is aiding Qaddafi and more... Sid hrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx CONFIDENTIAL March 3, 2011 For: Hillary From: Sid Re: Syria aiding Qaddafi This memo has two parts. Part one is the report that Syria is providing air support for Qaddafi. Part two is a note to Cody from Lord David Owen, former UK foreign secretary on his views of an increasingly complex crisis. It seems that the situation is developing into a protracted civil war with various nations backing opposing sides with unforeseen consequences. Under these circumstances the crucial challenge is to deprive Qaddafi of his strategic depth—his support both financial and military. I. Report During the afternoon of March 3, advisers to Muammar Qaddafi stated privately that the Libyan Leader has decided that civil war is inevitable, pitting troops and mercenary troops loyal to him against the rebel forces gathering around Benghazi. Qaddafi is convinced that these rebels are being supported by the United States, Western Europe and Israel. On March 2 Qaddafi told his son Saif al-Islam that he believes the intelligence services of the United States, Great Britain, Egypt, and France have deployed paramilitary officers to Benghazi to assist in organizing, training, and equipping opposition forces. Qaddafi is convinced that the National Libyan Council (NLC), and its leader, former Minister of Justice Mustafa Mohamed Abdel Galil have been chosen by the foreign powers to replace him. On March 1 advisors to Qaddafi stated that UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739546 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045000 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739546 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. Qaddafi's cousin, Col. Ali Qaddafiddam had failed in efforts to recruit fighters among the Egyptian population living immediately across the border with Libya. These individuals added that during the week of February 21 the Libyan Leader spoke to Syrian President Bashir al-Assad on at least three occasions by secure telephone lines. During the conversations Qaddafi asked that Syrian officers and technicians currently training the Libyan Air Force be placed under command of the Libyan Army and allowed to fight against the rebel forces. (Source Comment: Senior Libyan Army officers still loyal to Qaddafi added that On February 23, President Assad told General Isam Hallaq, the commander in chief of the Syrian Air Force, to instruct the pilots and technicians in Tripoli to help the Libyan regime, should full scale Civil War breaks out in the immediate future.) On March 2, a military officer with ties to Qaddfi's son Khamis stated privately that the number of Libyan pilots defecting to the opposition has destroyed the morale and professional spirit of the Libyan Air Force at this critical moment, when Tripoli's air superiority is its principal weapon against insurgents. In the opinion of this individual Qaddafi and his senior military advisors are convinced that the European Union and the U.S will impose a no-fly zone over Libya in the immediate future. These advisors believe that the no fly zone will serve as air support for opposition forces. They are also prepared for the Western allies to bomb anti-aircraft facilities in and around Tripoli in preparation for the establishment of the no-fly zone. Foreign Minister Mousa Kousa is convinced that that Russia and Turkey will oppose the move, and may prevent the implementation of the no fly zone. The Syrian soldiers in Libya are part of a mission established in 1984 following the signing of a military agreement between Qaddafi and Syria's long-time ruler and Bashir's father, Hafez al- Assad, in the presence of General Soubhi Haddad, who was the commander in chief of the Air Force at the time. Both Air Forces are equipped with Russian materiel and have had long- standing, close links with Moscow. Note from David Owen, former UK foreign secretary UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739546 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045001 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739546 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. In exchange for Syria's help, Libya provided financial support to the Syrian state, including funds in support of operations carried out by the Syrian intelligence services in Lebanon. Libyan money helped Hafez al-Assad bribe his brother Rifaat, the author of an attempted coup d'etat in 1983, to leave the country and go into exile in Spain and France, where he has remained ever since. According to individuals with access to the Syrian military, Damascus has also sent a second team of pilots and technicians to Tripoli. These are lower-ranking officers loyal to the regime who are specialized in flying helicopters. Before their departure for Tripoli on February 23, they met with General Allaq and General Jamil Hasan, head of Air Force's Intelligence. (Source Comment: During the afternoon of 3 March, an associate of Saif al-Islam Qaddafi stated that he and the Libyan leaders other family members were concerned over the announcement of Jose Luis Moreno-Ocampo, chief prosecutor of the International Criminal Court (ICC), announced that he was investigating Qaddafi, his sons Khamis, the commander of the 32 battalion, and Montasem, as well as the head of Gadhafi's personal security detail, the Director-General of the External Security Organization (Abuzaid Dorda), the spokesman of the regime (Musa Ibrahim), and the Minister of Foreign Affairs (Mousa Kousa). This source added that Saif al-Islam had said that Qaddafi himself found the announcement amusing.) From: Lord Owen Cody, Realistically the UN will not authorise a no fly zone while Gaddafi continues to hold off bombing and strafing but keeping it up front and on the military agenda keeps him worried. Yet for a few frustrating months that ambivalence kept the Serbian aeroplanes on the ground before we acted. My hope is that preparations continue with visuals of planes flying off carriers, airborne early warning planes flying and people with clout outside the Administration demanding preparations. But and it is a big BUT what else can and should we be doing? Encourage humanitarian ships and convoys from Egypt to bring in supplies since we must ensure the cities in the East can hold up living standards, maybe for months. More adventurous, Egyptian Special Forces to go in and advise, even supplying hand held missiles. It appears they have few missiles; the military in the East having been deliberately kept ill-equipped. There is old Nasserite sentiment for a Federation of Egypt, Sudan and Libya. I know some will say Egypt has enough problems and they should stay resolutely focused on domestic reforms. it is delicate but words alone and the balance of advantage will slip to Gaddafi. A Gaddafi victory is possible and needs to be weighed in the balance now when deciding what to do. I'm more worried than I am ready to say publicly. Yours David UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739546 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045002"
msg3 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739547 Date: 05/14/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B6 From: Mills, Cheryl D <MillsCD@state.gov> Sent: Wednesday, September 12, 2012 11:52 AM To: B6 Cc: Abedin, Huma Subject: Re: Chris Stevens Thx Original Message From: Anne-Marie Slaughter [ Sent: Wednesday, September 12, 2012 07:46 AM To: Ihdr22@clintonernail.com' <hdr22@clintonemail corn> Cc: Abed in, Huma; Mills, Cheryl D Subject: Chris Stevens To you and all my former colleagues, I'm so terribly sorry. Our FSOs stand on the front lines just as surely and devotedly as our soldiers do. Thinking of you and Pat and everyone this morning. AM UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739547 Date: 05/14/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045248"
msg4 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739550 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B5 From: Mills, Cheryl D <MillsCD@state.gov> Sent: Wednesday, September 12,2012 12:44 PM To: Subject: FVV: Cairo Condemnation - Final From: Smith, Dana S (PA) Sent: Wednesday, September 12, 2012 11:53 AM To Mills, Cheryl D Subject FW: Cairo Condemnation - Final B5 From: Snipe, Aaron D Sent: Tuesday, September 11 2012 1:31 PM To: Smith, Dana S (PA) Subject: FW: Cairo Condemnation Final FYI - From: Snipe, Aaron D Sent: Tuesday, September 11, 2012 6:18 AM To Schwartz, Larry; Ranz, David J; Sievers, Marc 3; Linfield, David E; Sayles, Ambrose G; NEA-Press-DL; NEA-Egypt; Shea, Peter T Cc: Mitchell, Andrew B Subject: RE: Cairo Condemnation - Final All, ADS Aaron D. Snipe Spokesperson I Bureau of Near Eastern Affairs I U.S. Department of State UNCLASSIFIF81 C Street NW I Room 6250 I Washington, DC, 20520 U.S. Department of State Case No. F-2015-04841 Doc No. C05739550 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. Date: 05/13/2015 SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045249 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739550 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. Office: +1 202-647-9533 I Email: snipead@state.gov From: Schwartz, Larry Sent Tuesday, September 11, 2012 5:26 AM To Ranz, David J; Sievers, Marc J; Linfield, David E; Sayles, Ambrose G; NEA-Press-DL; NEA-Egypt; Shea, Peter T cc: Mitchell, Andrew B Subject Cairo Condemnation - Final Importance: High B5 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739550 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045250"
msg5 = "B6 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B6 From: Sent: To: Subject: Attachments: H <hrod17@clintonemail.com> Friday, March 11, 2011 1:36 PM Huma Abedin Fw: H: Latest: How Syria is aiding Qaddafi and more... Sid hrc memo syria aiding libya 030311.docx Pis print. From: Sent: Thursday, March 03, 2011 09:45 PM To: H Subject: H: Latest: How Syria is aiding Qaddafi and more... Sid CONFIDENTIAL March 3, 2011 For: Hillary From: Sid Re: Syria aiding Qaddafi This memo has two parts. Part one is the report that Syria is providing air support for Qaddafi. Part two is a note to Cody from Lord David Owen, former UK foreign secretary on his views of an increasingly complex crisis. It seems that the situation is developing into a protracted civil war with various nations backing opposing sides with unforeseen consequences. Under these circumstances the crucial challenge is to deprive Qaddafi of his strategic depth—his support both financial and military. I. Report During the afternoon of March 3, advisers to Muammar Qaddafi stated privately that the Libyan Leader has decided that civil war is inevitable, pitting troops and mercenary troops loyal to him against the rebel forces gathering around Benghazi. Qaddafi is convinced that these rebels are being supported by the United States, Western Europe and Israel. On March 2 Qaddafi told his son Saif al-Islam that he believes the intelligence services of the United States, Great Britain, UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045003 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. Egypt, and France have deployed paramilitary officers to Benghazi to assist in organizing, _raining, and equipping opposition forces. Qaddafi is convinced that the National Libyan Council :NLC), and its leader, former Minister of Justice Mustafa Mohamed Abdel Gall have been :thosen by the foreign powers to replace him. On March 1 advisors to Qaddafi stated that Q,addafi's cousin, Col. Al Qaddafiddam had failed in efforts to recruit fighters among the Egyptian population living immediately across the border with Libya. These individuals added that during the week of February 21 the Libyan Leader spoke to Syrian president Bashir al-Assad on at least three occasions by secure telephone lines. During the :,onversations Qaddafi asked that Syrian officers and technicians currently training the Libyan /kir Force be placed under command of the Libyan Army and allowed to fight against the rebel tbrces. (Source Comment: Senior Libyan Army officers still loyal to Qaddafi added that On February 23, President Assad told General Isam HaIlaq, the commander in chief of the Syrian Air Force, :o instruct the pilots and technicians in Tripoli to help the Libyan regime, should full scale Civil War breaks out in the immediate future.) On March 2, a military officer with ties to Qaddfi's son Khamis stated privately that the number Df Libyan pilots defecting to the opposition has destroyed the morale and professional spirit of ,he Libyan Air Force at this critical moment, when Tripoli's air superiority is its principal weapon against insurgents. In the opinion of this individual Qaddafi and his senior military advisors are convinced that the European Union and the U.S will impose a no-fly zone over Libya in the immediate future. These advisors believe that the no fly zone will serve as air support for Dpposition forces. They are also prepared for the Western allies to bomb anti-aircraft facilities in and around Tripoli in preparation for the establishment of the no-fly zone. Foreign Minister Mousa Kousa is convinced that that Russia and Turkey will oppose the move, and may prevent the implementation of the no fly zone. The Syrian soldiers in Libya are part of a mission established in 1984 following the signing of a military agreement between Qaddafi and Syria's long-time ruler and Bashir's father, Hafez al- UNCLASSIFIED IL from David Owen, former UK foreign secretary U.S. Department of State Case No F-2015-04841 Doc No C05739554 STATE DEPT - PRODUCED TO HOUSE SELECT BENGHAZI COMM Date 05/13/2015 SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS NO FOIA WAIVER STATE-SCB0045004 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. kssad, in the presence of General Soubhi Haddad, who was the commander in chief of the Air Force at the time. Both Air Forces are equipped with Russian materiel and have had long- ;tanding, close links with Moscow. :n exchange for Syria's help, Libya provided financial support to the Syrian state, including funds n support of operations carried out by the Syrian intelligence services in Lebanon. Libyan money -wiped Hafez al-Assad bribe his brother Rifaat, the author of an attempted coup d'etat in 1983, .o leave the country and go into exile in Spain and France, where he has remained ever since. kccording to individuals with access to the Syrian military, Damascus has also sent a second earn of pilots and technicians to Tripoli. These are lower-ranking officers loyal to the regime Afho are specialized in flying helicopters. Before their departure for Tripoli on February 23, they net with General Allaq and General Jamil Hasan, head of Air Force's Intelligence. :Source Comment: During the afternoon of 3 March, an associate of Saif al-Islam Qaddafi ;tated that he and the Libyan leaders other family members were concerned over the trmouncement of Jose Luis Moreno-Ocampo, chief prosecutor of the International Criminal .2ourt (ICC), announced that he was investigating Qaddafi, his sons Khamis, the commander of .he 32 battalion, and Montasem, as well as the head of Gadhafi's personal security detail, the Director-General of the External Security Organization (Abuzaid Dorda), the spokesman of the -egime (Musa Ibrahim), and the Minister of Foreign Affairs (Mousa Kousa). This source added :hat Saif al-Islam had said that Qaddafi himself found the announcement amusing.) From: Lord Owen Cody, Realistically the UN will not authorise a no fly zone while Gaddafi continues to hold off bombing and strafing but keeping it up front and on the military agenda keeps him worried. Yet for a few frustrating months that ambivalence kept the Serbian aeroplanes on the ground before we acted. My hope is that preparations continue with visuals of planes flying off carriers, airborne early warning planes flying and people with clout outside the Administration demanding preparations. But and it is a big BUT what else can and should we be doing? Encourage humanitarian ships and convoys from Egypt to bring in supplies since we must ensure the cities in the East can hold up living standards, maybe for months. More adventurous, Egyptian Special Forces to go in and advise, even supplying hand held missiles. It appears they have few missiles; the military in the East having been deliberately kept ill-equipped. There is old Nasserite sentiment for a Federation of Egypt, Sudan and Libya. I know some will say Egypt has enough problems and they should stay resolutely focused on domestic reforms. it is delicate but words alone and the balance of advantage will slip to Gaddafi. A Gaddafi UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. Date: 05/13/2015 SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045005 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. victory is possible and needs to be weighed in the balance now when deciding what to do. Pm more worried than I am ready to say publicly. Yours David UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739554 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045006"
msg6 = "B6 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739559 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B6 From: Sent To: Subject Pis print. -•-...-^ H < hrod17@clintonernailcom> Wednesday, September 12, 2012 2:11 PM °Russorv@state.gov' Fw: Meet The Right-Wing Extremist Behind Anti-fvluslim Film That Sparked Deadly Riots From [meat) Sent: Wednesday, September 12, 2012 01:00 PM To: 11 Subject: Meet The Right Wing Extremist Behind Anti-Muslim Film That Sparked Deadly Riots htte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked- deadly-riots/ Sent from my Verizon Wireless 4G LTE DROID UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739559 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045251"


In [243]:
msg7 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739560 \
Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT \
TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN \
FULL From: Mills, Cheryl D <MillsCD@state.gov> Sent: Wednesday, September 12, \
2012 4:00 PM To: Subject: FW: Anti-Muslim film director in hiding, following \
Libya, Egypt violence From: Toiv, Nora F Sent: Wednesday, September 12, 2012 \
1:48 PM To: Mills, Cheryl D Subject: Anti-Muslim film director in hiding, following \
Libya, Egypt violence Anti-Muslim film director in hiding, following Libya, Egypt \
violence By Nia-Malika Henderson and Michelle Boorstein, Updated: Wednesday, \
September 12, 1:30 PM The director of a controversial anti-Islam film has gone \
into hiding, according to reports, fearing reprisals over a work that has sparked \
violence in Egypt and Libya and led to the deaths of four Americans. \
Sam Bacile, a California real estate developer, posted a 13-minute clip \
of \"The Innocence of Muslims,\" in July, and the film became a lighting \
rod after the Egyptian media began showing parts of it on air and dubbed \
versions of the English-language film appeared on the Internet. It depicts \
the Prophet Muhammad as a bumbling idiot, born out of wedlock and making up \
verses to the Islamic holy book to suit his purposes and desires. The film \
also shows him as having intimate relations with women and suggests that he \
was gay. Any flesh-and-blood depiction of Muhammad is offensive to\
Muslims. \"This is a political movie,\" Bacile, 56, told the Associated Press, \
adding that he didn\'t expect the film to spark such \
fury. \"The U.S. lost a lot of money and a lot of people in wars in Iraq and \
Afghanistan, but we're fighting with ideas.\" Bacile identified himself \
to several news organizations as an Israeli-born Jew who works as a real \
estate developer in California. Yet his identity remains something of a \
mystery—Bacile is not listed in any directories or incorporations or real \
estate deeds and is not licensed in California as a real estate broker. \
And Israeli authorities told the Associated Press that they don\'t have \
any records of Bacile. UNCLASSIFIED U.S. Department of State Case No. \
F-2015-04841 Doc No. C05739560 Date: 05/13/2015 STATE DEPT. - PRODUCED \
TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE \
INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045252 UNCLASSIFIED \
U.S. Department of State Case No. F-2015-04841 Doc No. C05739560 \
Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI\
COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. \
NO FOIA WAIVER. There are also questions about whether the film sparked the \
violence in Libya, which according to some reports seemed to be more \
coordinated than the protests in Egypt where some 2,000 gathered outside \
the U.S. Embassy in Cairo. The eruption of violence came around the llth \
anniversary of the Sept. 11 terror attacks, as Americans paused to remember \
the 3,000 people who lost their lives. Ambassador J. Christopher Stevens and \
three others were killed after protesters attacked the U.S. consulate in Benghazi.\
Nihad Awad, national executive director of the Washington-based Council on \
American-Islamic Relations (CAIR), condemned both the killings and the film. \
Awad said in a statement: \"We urge that this ignorant attempt to provoke the \
religious feelings of Muslims in the Arabic-speaking world be ignored and \
that its extremist producers not be given the cheap publicity they so desperately \
seek. \"Those who created this trashy film do not represent the people of America \
or the Christian faith. The only proper response to intentional provocations such \
as this film is to redouble efforts to promote mutual understanding between faiths \
and to marginalize extremists of all stripes.\" A controversial Florida pastor \
was among those promoting \"The Innocence of Muslims,\" which cost $5 million to \
make and stars amateur actors, some of whom appear to be wearing make-up to darken \
their faces. Bacile said that the film was backed by 100 Jewish donors, according to \
Bacile. Among the promoters is Morris Sadek, who heads the National American Cooptic \
Assembly and the Rev. Terry Jones, leader of a small group that holds virulently \
anti-Islam events. Tuesday evening, Jones sent out a press release blaming the Muslim faith for the protests and saying the group would be running a trailer later in the evening for the film. The trailer was part of a one-to-two hour \"live event\" that Jones was doing from his Gainesville center for Sept. 11, said Fran Ingram, a spokeswoman for his group, Stand Up America. The whole broadcast included the handlebar-moustached pastor in a black \"infidel\" t-shirt standing in front of a fabric lynched devil dressed in a white turban for what they group dubbed \"International Judge Muhammed Day.\" It wasn\'t clear if Jones, who garnered worldwide attention in recent years with his threats to burn the Koran, had promoted \"The Innocence of Muslims\" before the rioting began. Asked if Jones was connected in any way with the making of the film, Ingram said she couldn\'t comment. Alice Crites contributed to this story. © The Washington Post Company UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739560 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045253"

In [246]:
msg8 = "B6 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B6 From: Sent: To: Subject: Attachments: H <hrod17@clintonemail.corn> Friday, March 11, 2011 1:36 PM Huma Abedin Fw: H: Latest: How Syria is aiding Qaddafi and more... Sid hrc memo Syria aiding libya 030311.docx Pis print. From: Sent: Thursday, March 03, 2011 09:45 PM To: H Subject: H: Latest: How Syria is aiding Qaddafi and more... Sid CONFIDENTIAL March 3, 2011 For: Hillary From: Sid Re: Syria aiding Qaddafi This memo has two parts. Part one is the report that Syria is providing air support for Qaddafi. Part two is a note to Cody from Lord David Owen, former UK foreign secretary on his views of an increasingly complex crisis. It seems that the situation is developing into a protracted civil war with various nations backing opposing sides with unforeseen consequences. Under these circumstances the crucial challenge is to deprive Qaddafi of his strategic depth--his support both financial and military. I. Report During the afternoon of March 3, advisers to Muammar Qaddafi stated privately that the Libyan Leader has decided that civil war is inevitable, pitting troops and mercenary troops loyal to him against the rebel forces gathering around Benghazi. Qaddafi is convinced that these rebels are being supported by the United States, Western Europe and Israel. On March 2 Qaddafi told his son Saif al-Islam that he believes the intelligence services of the United States, Great Britain, UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045007 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. Egypt, and France have deployed paramilitary officers to Benghazi to assist in organizing, training, and equipping opposition forces. Qaddafi is convinced that the National Libyan Council (NLC), and its leader, former Minister of Justice Mustafa Mohamed Abdel Galil have been chosen by the foreign powers to replace him. On March 1 advisors to Qaddafi stated that Qaddafi's cousin, Col. Ali Qaddafiddam had failed in efforts to recruit fighters among the Egyptian population living immediately across the border with Libya. These individuals added that during the week of February 21 the Libyan Leader spoke to Syrian President Bashir al-Assad on at least three occasions by secure telephone lines. During the conversations Qaddafi asked that Syrian officers and technicians currently training the Libyan Air Force be placed under command of the Libyan Army and allowed to fight against the rebel forces. (Source Comment: Senior Libyan Army officers still loyal to Qaddafi added that On February 23, President Assad told General Isam HaIlaq, the commander in chief of the Syrian Air Force, to instruct the pilots and technicians in Tripoli to help the Libyan regime, should full scale Civil War breaks out in the immediate future.) On March 2, a military officer with ties to Qaddfi's son Khamis stated privately that the number of Libyan pilots defecting to the opposition has destroyed the morale and professional spirit of the Libyan Air Force at this critical moment, when Tripoli's air superiority is its principal weapon against insurgents. In the opinion of this individual Qaddafi and his senior military advisors are convinced that the European Union and the U.S will impose a no-fly zone over Libya in the immediate future. These advisors believe that the no fly zone will serve as air support for opposition forces. They are also prepared for the Western allies to bomb anti-aircraft facilities in and around Tripoli in preparation for the establishment of the no-fly zone. Foreign Minister Mousa Kousa is convinced that that Russia and Turkey will oppose the move, and may prevent the implementation of the no fly zone. The Syrian soldiers in Libya are part of a mission established in 1984 following the signing of a military agreement between Qaddafi and Syria's long-time ruler and Bashir's father, Hafez al- UNCLASSIFIED Note from David Owen, former UK foreign secretary U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. Date: 05/13/2015 SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045008 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. Assad, in the presence of General Soubhi Haddad, who was the commander in chief of the Air Force at the time. Both Air Forces are equipped with Russian materiel and have had long- standing, close links with Moscow. In exchange for Syria's help, Libya provided financial support to the Syrian state, including funds in support of operations carried out by the Syrian intelligence services in Lebanon. Libyan money helped Hafez al-Assad bribe his brother Rifaat, the author of an attempted coup d'etat in 1983, to leave the country and go into exile in Spain and France, where he has remained ever since. According to individuals with access to the Syrian military, Damascus has also sent a second team of pilots and technicians to Tripoli. These are lower-ranking officers loyal to the regime who are specialized in flying helicopters. Before their departure for Tripoli on February 23, they met with General Allaq and General Jamil Hasan, head of Air Force's Intelligence. (Source Comment: During the afternoon of 3 March, an associate of Saif al-Islam Qaddafi stated that he and the Libyan leaders other family members were concerned over the announcement of Jose Luis Moreno-Ocampo, chief prosecutor of the International Criminal Court (ICC), announced that he was investigating Qaddafi, his sons Khamis, the commander of the 32 battalion, and Montasem, as well as the head ofGadhaffs personal security detail, the Director-General of the External Security Organization (Abuzaid Dorda), the spokesman of the regime (Musa Ibrahim), and the Minister of Foreign Affairs (Mousa Kousa). This source added that Saif al-Islam had said that Qaddafi himself found the announcement amusing.) From: Lord Owen Cody, Realistically the UN will not authorise a no fly zone while Gaddafi continues to hold off bombing and strafing but keeping it up front and on the military agenda keeps him worried. Yet for a few frustrating months that ambivalence kept the Serbian aeroplanes on the ground before we acted. My hope is that preparations continue with visuals of planes flying off carriers, airborne early warning planes flying and people with clout outside the Administration demanding preparations. But and it is a big BUT what else can and should we be doing? Encourage humanitarian ships and convoys from Egypt to bring in supplies since we must ensure the cities in the East can hold up living standards, maybe for months. More adventurous, Egyptian Special Forces to go in and advise, even supplying hand held missiles. It appears they have few missiles; the military in the East having been deliberately kept ill-equipped. There is old Nasserite sentiment for a Federation of Egypt, Sudan and Libya. I know some will say Egypt has enough problems and they should stay resolutely focused on domestic reforms. it is delicate but words alone and the balance of advantage will slip to Gaddafi. A Gaddafi UNCLASSIFIED U.S. Department of State Case No F-2015-04841 Doc No C05739561 STATE DEPT - PRODUCED TO HOUSE SELECT BENGHAZI COMM Date 05/13/2015 SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS NO FOIA WAIVER STATE-SCB0045009 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. victory is possible and needs to be weighed in the balance now when deciding what to do. Pm more worried than I am ready to say publicly. Yours David UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739561 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045010"
msg9 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739562 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN FULL From: Sullivan, Jacob J <Sullivanli@stategov> Sent: Wednesday, September 12, 2012 6:08 PM To: Subject: FVV: Secretary's remarks FYI From: Walsh, Matthew P Sent: Wednesday, September 12, 2012 5:37 PM To Daniel, Joshua 3; Benaim, Daniel; Rooney, Megan; Schwerin, Daniel B Subject AN Secretary's remarks Don't know if you guys have seen this, but there is an article on Slate right now that describes the below as \"her most eloquent news conference as Secretary of State.\" Really nice work guys. htto://www.slate.comiarticlesinews arid politics/war stories/2012/09/libya u s embassy attack hillarv clinton was eloquent regarding ambassador stevens mitt romnev smirked ,htrni From: McAuliffe, Marisa S Sent: Wednesday, September 12, 2012 5:31 PM To: SPoExpandecl Subject: Secretary's remarks For anyone who missed the Secretary's remarks this morning, they are worth a read: htto://www,state.govisecretary/rm/2012/09/197654.htm Remarks on the Deaths of American Personnel in Benghazi, Libya Remarks Hillary Rodham Clinton Secretary of State Treaty Room Washington, DC September 12, 2012 Yesterday, our U.S. diplomatic post in Benghazi, Libya was attacked. Heavily armed militants assaulted the compound and set fire to our buildings, American and Libyan security personnel battled the attackers together, Four Americans were killed, They included Sean Smith, a Foreign Service information management officer, and our Ambassador to Libya Chris Stevens. We are still making next of kin notifications for the other two individuals. This is an attack that should shock the conscience of people of all faiths around the vvorld. We condemn in the strongest terms this senseless act of violence, and we send our prayers to the families, friends, and colleagues of those we've lost. All over the world, every day, America's diplomats and development experts risk their lives in the service of our country and our values, because they believe that the United States must be a force for peace and progress in the world, that UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739562 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045254 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739562 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. these aspirations are worth striving and sacrificing for. Alongside our men and women in uniform, they represent the best traditions of a bold and generous nation. In the lobby of this building, the State Department, the names of those who have fallen in the line of duty are inscribed in marble. Our hearts break over each one. And now, because of this tragedy, we have new heroes to honor and more friends to mourn. Chris Stevens fell in love with the Middle East as a young Peace Corps volunteer teaching English in Morocco. He joined the Foreign Service, learned languages, won friends for America in distant places, and made other people's hopes his own. In the early days of the Libyan revolution, I asked Chris to be our envoy to the rebel opposition. He arrived on a cargo ship in the port of Benghazi and began building our relationships with Libya's revolutionaries. He risked his life to stop a tyrant, then gave his life trying to help build a better Libya. The world needs more Chris Stevenses. I spoke with his sister, Ann, this morning, and told her that he will be remembered as a hero by many nations. Sean Smith was an Air Force veteran. He spent 10 years as an information management officer in the State Department, he was posted at The Hague, and was in Libya on a brief temporary assignment. He was a husband to his wife Heather, with whom I spoke this morning. He was a father to two young children, Samantha and Nathan. They will grow up being proud of the service their father gave to our country, service that took him from Pretoria to Baghdad, and finally to Benghazi. The mission that drew Chris and Sean and their colleagues to Libya is both noble and necessary, and we arid the people of Libya honor their memory by carrying it forward. This is not easy. Today, many Americans are asking indeed, I asked myself — how could this happen? How could this happen in a country we helped liberate, in a city we helped save from destruction? This question reflects just how complicated and, at times, how confounding the world can be But we must be clear-eyed, even in our grief. This was an attack by a small and savage group — not the people or Government of Libya. Everywhere Chris and his team went in Libya, in a country scarred by war and tyranny, they were hailed as friends and partners. And when the attack came yesterday, Libyans stood and fought to defend our post, Some were wounded. Libyans carried Chris' body to the hospital, and they helped rescue and lead other Americans to safety. And last night, when I spoke with the President of Libya, he strongly condemned the violence and pledged every effort to protect our people and pursue those responsible. The friendship between our countries, borne out of shared struggle, will not be another casualty of this attack. A free and stable Libya is still in America's interest and security, and we will not turn our back on that nor will we rest until those responsible for these attacks are found and brought to justice. We are working closely with the Libyan authorities to move swiftly and surely. We are also working with partners around the world to safeguard other American embassies, consulates, and citizens. There will be more time later to reflect, but today, we have work to do. There is no higher priority than protecting our men and women wherever they serve. We are working to determine the precise motivations and methods of those who carried out this assault. Some have sought to justify this vicious behavior, along with the protest that took place at our Embassy in Cairo yesterday, as a response to inflammatory material posted on the Internet. America's commitment to religious tolerance goes back to the very beginning of our nation. But let me be clear —there is no justification for this, none. Violence like this is no way to honor religion or faith. And as long as there are those who would take innocent life in the name of God, the world will never know a true and lasting peace. UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739562 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045255 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739562 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. It is especially difficult that this happened on September 11th. It's an anniversary that means a great deal to all Americans. Every year on that day, we are reminded that our work is not yet finished, that the job of putting an end to violent extremism and building a safe and stable world continues. But September 11th means even more than that. It is a day on which we remember thousands of American heroes, the bonds that connect all Americans, wherever we are on this Earth, and the values that see us through every storm. And now it is a day on which we will remember Sean, Chris, and their colleagues. May God bless them, and may God bless the thousands of Americans working in every corner of the world who make this country the greatest force for peace, prosperity, and progress, and a force that has always stood for human dignity — the greatest force the world has ever known. And may God continue to bless the United States of America. Thank you. UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739562 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045256"
msg10 = "UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN PART B6 From: Sent: To: Subject Attachments: B6 Wednesday, September 12, 2012 6:16 PM Fwd: more on libya Libya 37 sept 12 12.docx Sending direct. Just in. Sent from my Verizon Wireless 4G LTE DRUID Original Message Subject: more on Libya From: tyler drumheller &It: igt; To &It- Fcgt: CC: B6 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045257 UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. . PROPRIETARY/CONFIDENTIAL SUBJECT: Libya (37) SOURCE: Sources with direct access to the Libyan National Transitional Council, as well as the highest levels of European Governments, and Western Intelligence and security services. THE FOLLOWING INFORMATION COMES FROM AN EXTREMELY SENSITIVE SOURCE AND SHOULD BE HANDLED WITH CARE. 1. On September 12, 2012 Libyan President Mohammed Yussef el Magariaf told senior advisors that the death of the U.S. Ambassador at the hands of Islamist militia forces represents a threat to the future of the newly elected General National Congress (GNC) Government. According to a sensitive source, el Magariaf believes that the primary goal of this and other attacks on Western facilities is to demonstrate that the GNC cannot protect its non-Islamic friends. Libyan security officials believe that the attack was carried out by forces of the Islamist militia group calling itself the Ansar al Sharia brigade; working out of camps in the Eastern suburbs of Benghazi. These officials added that their sources indicate that this group is led by former members of the old Libyan Islamic Fighting Group (LIFG), some of whom are loyal to Islamist General Abdelhakim Belhaj. The same officials were quick to point out that they have no evidence that Belhaj had any direct involvement in this attack. They also have no direct information as to whether this assault has anything to do with the death of Libyan born al Qai'da leader Abu Yahya al- Libi, but they are pursuing this theme. These officials do UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045258 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. believe that the attackers having prepared to launch their assault took advantage of the cover provided by the demonstrations in Benghazi protesting an intemet production seen as disrespectful to the prophet Mohammed. 2. According to this source, the immediate events were set in motion by a statement made by a Muslim Cleric in Egypt saying that the internet film was going to be shown across the United States on September 11 in an effort to insult Muslims on the anniversary of the attacks on the New York World Trade Center in 2001. This statement inspired increasingly hostile demonstrations at the U.S. Embassy in Cairo, and at diplomatic facilities in Libya. In Benghazi, reporting from security sources states, approximately twenty one (21) Ansar al Sharia fighters left their base in East Benghazi just after sundown infiltrated the crowd of about 2,000 demonstrators at the U.S. Consulate. These forces, operating under cover of darkness, opened fire on the consulate, eventually setting it on fire with rocket propelled grenades. These fighters then withdrew to their camp. El Magariaf ordered security officials to identify and arrest the attackers, adding that he had been in touch with Egyptian President Mohm_med Morsi, and the two agreed that their intelligence and security personnel will cooperate in this matter. (Note: Some of the Libyan officials believe that the entire demonstration was organized as cover for the attack, however; they point out that there is no evidence of such a complex operation at this time. 3.(Source Comment: In the opinion of this individual, Libyan security officers also informed el Magariaf that the attacks had been planned for approximately one month, based on casing information obtained during an early demonstration at the U.S. consulate in Benghazi. The attackers were, in the opinion of these individuals, looking for an opportunity to approach the consulate UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045259 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. under cover in a crowd. These officials add that government security officers admitted to el Magariaf that they do not have the physical capacity to locate, attack, and destroy the Ansar al-Sharia brigade, among others. They describe these forces as well-trained, hardened killers; many of whom have spent time in Afghanistan and Yemen. These fighters are located in camps in Eastern Libya; and Libyan officials doubt they can penetrate these camps because the militia's are better armed than the forces of the new National Army. 4. In the opinion of a separate source El Magariaf continues to believe that the attacks were planned and carried out in a political atmosphere where a portion of the Libyan population is focused on past cooperation between Western intelligence and security services, and the regime of former dictator Muammar al Qaddafi. According to this source, el Magariaf realizes that the death of the U.S. Ambassador dictates that he must find tangible solutions to the problems tied to disarming the militias, dealing Salafist violence against the followers of Sufi beliefs, and efforts to discredit his government through linking him to Western intelligence services. El Magariaf instructed his security officials to cooperate with their U.S. counterparts. He also warned all of his advisors to be prepared for U.S. retaliation against the suspected perpetrators of this attack, and the angry public reaction that is sure to follow. 5. (Source Comment: A separate, particularly sensitive source added that in private conversation el Magariaf stated that if his government cannot protect a senior diplomatic official, foreign businesses will not be willing to risk opening up new operations in Libya. According to this sensitive source, el Magariaf believes that a number of the Islamist militias are now being influenced al Qai'da and other radical groups; working out of camps in the Eastern and Southeastern parts of the UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045260 \
UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. country. These groups are receiving aid and weapons from supporters in Sudan. In the end, el Magariaf added that while he does not expect U.S. officials to focus on this point, he believes the attacks against the interests and citizens of the U.S. and other Western states are aimed first at overthrowing his government, and replacing it with a strict Islamic Republic dominated by Salafist leaders. Magariaf also warned that tribal militias loyal to the Qaddaft family continue to present an equally dangerous threat to the new Government.) CONFIDENTIAL: This message is confidential, privileged, and is covered by the Electronic Communications Privacy Act (18 USC 2510). This message and any attachments is intended for the sole use of the addresses(s) and any discussion, copying and/or distribution is strictly prohibited. If you have received this message in error, please notify the sender, and immediately delete from your computer system. Thank you. UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739563 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045261"

In [ ]:
# extract subject from message
l = [msg1,msg2,msg3,msg4,msg5,msg6,msg7,msg8,msg9, msg10]
for m in l:
    

In [426]:
test_list = topic_modeling_df['RawText'].tolist()#[msg1, msg2, msg3, msg4, msg5, msg6, msg7, msg8, msg9, msg10]            
type(test_list)

list

In [428]:
# the raw text of the document is split into sentences using a sentence segmenter

tokens_per_msg = []
re_longer_than_2 = re.compile('^[a-z]{3,}')

for msg in test_list:
    sentences = nltk.sent_tokenize(msg)
    del sentences[:6]
    del sentences[-7:]
    
    tokens = []
    for s in sentences:
        curr_tokens = nltk.word_tokenize(s)
        
        curr_tokens  = [word for word in curr_tokens if word in english_vocab]
        curr_tokens = [word for word in curr_tokens if word not in stopwords]
        curr_tokens = list(filter(lambda x: re_longer_than_2.match(x) ,curr_tokens))
        tokens = tokens+curr_tokens

    tokens_per_msg.append(tokens)

In [439]:
# Stemming 
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# stem token
#stemmed_tokens_per_msg = [[p_stemmer.stem(t) for t in tokens] for tokens in tokens_per_msg]
stemmed_tokens_per_msg = tokens_per_msg

for tokens in stemmed_tokens_per_msg:
    print(tokens, '\n')
    break

['memo', 'memo', 'memo', 'two', 'one', 'report', 'providing', 'air', 'support', 'two', 'note', 'former', 'foreign', 'secretary', 'increasingly', 'complex', 'crisis', 'situation', 'protracted', 'civil', 'war', 'various', 'backing', 'opposing', 'sides', 'unforeseen', 'crucial', 'challenge', 'deprive', 'strategic', 'support', 'financial', 'military', 'afternoon', 'stated', 'privately', 'decided', 'civil', 'war', 'inevitable', 'pitting', 'mercenary', 'loyal', 'rebel', 'gathering', 'around', 'convinced', 'told', 'son', 'intelligence', 'paramilitary', 'assist', 'training', 'opposition', 'convinced', 'leader', 'former', 'chosen', 'foreign', 'replace', 'stated', 'cousin', 'recruit', 'among', 'population', 'living', 'immediately', 'across', 'border', 'added', 'week', 'spoke', 'least', 'three', 'secure', 'telephone', 'currently', 'training', 'command', 'fight', 'rebel', 'still', 'loyal', 'added', 'told', 'commander', 'chief', 'instruct', 'help', 'regime', 'full', 'scale', 'immediate', 'future', 

### Constructing a document-term matrix

In [440]:
# To generate an LDA model, we need to understand how frequently each term occurs within each document. 
# To do that, we need to construct a document-term matrix with a package called gensim:
from gensim import corpora, models
dictionary = corpora.Dictionary(stemmed_tokens_per_msg)

In [441]:
#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

In [442]:
# The Dictionary() function traverses texts, assigning a unique integer id to each unique token 
# while also collecting word counts and relevant statistics. 
# To see each token’s unique integer id, try print(dictionary.token2id).
print(list(dictionary.token2id.items())[:10])

[('assailant', 0), ('opener', 2), ('countermand', 3), ('unload', 4), ('tailoring', 5), ('turbulence', 6), ('migration', 7), ('suspected', 8), ('myth', 9), ('disparaging', 10)]


In [443]:
# Next, our dictionary must be converted into a bag-of-words:
corpus = [dictionary.doc2bow(tokens) for tokens in stemmed_tokens_per_msg]

In [444]:
# The doc2bow() function converts dictionary into a bag-of-words. 
# The result, corpus, is a list of vectors equal to the number of documents. 
# In each document vector is a series of tuples. As an example, print(corpus[0]) results in the following:
print(corpus[0])

[(123, 7), (165, 3), (206, 1), (235, 1), (286, 1), (294, 1), (310, 2), (356, 1), (374, 1), (391, 1), (471, 3), (596, 1), (682, 2), (690, 1), (711, 2), (724, 1), (786, 1), (950, 1), (954, 1), (983, 4), (1057, 1), (1156, 1), (1211, 1), (1335, 1), (1423, 1), (1478, 1), (1524, 1), (1599, 2), (1608, 1), (1664, 1), (1715, 1), (1854, 1), (1899, 1), (1990, 1), (2060, 2), (2063, 1), (2064, 1), (2113, 1), (2150, 1), (2196, 2), (2201, 3), (2213, 1), (2271, 2), (2323, 1), (2332, 1), (2334, 1), (2388, 1), (2400, 1), (2438, 1), (2478, 1), (2644, 1), (2674, 1), (2695, 1), (2899, 2), (2912, 1), (2928, 1), (2977, 1), (3167, 1), (3198, 3), (3243, 1), (3279, 1), (3327, 1), (3350, 3), (3351, 2), (3367, 1), (3667, 1), (3738, 3), (3746, 1), (3777, 1), (3840, 1), (3951, 2), (4019, 2), (4097, 2), (4130, 1), (4283, 1), (4337, 1), (4349, 1), (4424, 1), (4593, 1), (4608, 1), (4646, 1), (4663, 1), (4758, 1), (4849, 1), (4899, 1), (4901, 1), (4904, 2), (4926, 1), (4977, 1), (4991, 1), (5008, 1), (5018, 2), (5062, 

This list of tuples represents the first message. The tuples are (term ID, term frequency) pairs. doc2bow() only includes terms that actually occur: terms that do not occur in a document will not appear in that document’s vector.

### Applying the LDA model

In [454]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, passes=20)

### Examining the results

In [455]:
print(ldamodel.print_topics(num_topics=5, num_words=3))
ldamodel.show_topics()

[(29, '0.138*"per" + 0.110*"cent" + 0.022*"public"'), (26, '0.034*"election" + 0.033*"party" + 0.028*"would"'), (17, '0.016*"said" + 0.014*"reconstruction" + 0.012*"local"'), (21, '0.014*"military" + 0.013*"would" + 0.011*"former"'), (23, '0.036*"update" + 0.025*"family" + 0.020*"time"')]


[(45,
  '0.025*"people" + 0.024*"get" + 0.020*"plan" + 0.019*"could" + 0.018*"kids" + 0.018*"school" + 0.015*"education" + 0.014*"work" + 0.012*"though" + 0.011*"help"'),
 (40,
  '0.044*"percent" + 0.022*"said" + 0.020*"enough" + 0.017*"would" + 0.015*"need" + 0.012*"margin" + 0.012*"climate" + 0.011*"opinion" + 0.010*"help" + 0.010*"change"'),
 (4,
  '0.026*"shall" + 0.022*"train" + 0.020*"nomination" + 0.016*"would" + 0.016*"turnout" + 0.016*"coup" + 0.014*"hold" + 0.014*"lift" + 0.014*"candidate" + 0.014*"recommend"'),
 (19,
  '0.027*"staff" + 0.021*"president" + 0.017*"adviser" + 0.017*"former" + 0.016*"chief" + 0.015*"secretary" + 0.015*"earthquake" + 0.014*"year" + 0.013*"team" + 0.011*"campaign"'),
 (37,
  '0.017*"new" + 0.016*"policy" + 0.013*"long" + 0.012*"video" + 0.012*"growth" + 0.012*"bank" + 0.011*"like" + 0.011*"society" + 0.009*"foreign" + 0.009*"ever"'),
 (33,
  '0.043*"aid" + 0.033*"money" + 0.026*"government" + 0.024*"tax" + 0.022*"speech" + 0.019*"spending" + 0.019

In [453]:
topics_matrix = ldamodel.show_topics(formatted=False, num_words=20)
for i, words in topics_matrix:
    print([w for w, p in words], '\n')

['movement', 'group', 'right', 'company', 'conservative', 'police', 'cable', 'white', 'front', 'oil', 'race', 'one', 'onto', 'death', 'hill', 'liberal', 'media', 'left', 'black', 'campaign'] 

['percent', 'would', 'religious', 'world', 'treaty', 'nation', 'new', 'faith', 'government', 'long', 'economic', 'like', 'growth', 'one', 'many', 'religion', 'debt', 'toward', 'face', 'political'] 

['deal', 'said', 'party', 'agreement', 'would', 'assembly', 'statement', 'last', 'new', 'peace', 'two', 'justice', 'vote', 'leader', 'see', 'support', 'press', 'community', 'government', 'minister'] 

['call', 'tomorrow', 'time', 'know', 'morning', 'today', 'back', 'update', 'office', 'like', 'route', 'speak', 'work', 'confirmed', 'schedule', 'talk', 'let', 'memo', 'available', 'need'] 

['administration', 'policy', 'nuclear', 'foreign', 'president', 'world', 'new', 'strategy', 'public', 'diplomacy', 'war', 'time', 'peace', 'would', 'state', 'official', 'even', 'one', 'settlement', 'senior'] 

['say',